In [1]:
import requests
from urllib.parse import quote

client_id = "SWe77FM8c_EjAm5Y93h5"
client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=10):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# if 'items' in result:
#   for item in result['items']:
#     if item['brand'] != '':
#       print(f"브랜드: {item['brand']}")
#       print(f"이름: {item['title']}")
#       print(f"상품ID: {item['productId']}")
#       print(f"링크: {item['link']}")
#       print("-" * 50)
# else:
#   print("Error:", result.get('error', 'Unknown error'))


In [2]:
!pip install -q chromadb sentence-transformers
!pip install -q langchain langchain-community langchain-core langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 8.7 MB/s eta 0:

In [3]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 1. 데이터 전처리 함수
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# 2. Chroma DB 설정
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,  # 반드시 임베딩 함수 재지정
    metadata={"hnsw:space": "cosine"}  # 거리 측정 방식 변경
)

# 3. 데이터 삽입 (API 결과 사용)
if 'items' in result:
    processed_data = preprocess_items(result['items'])

    # 벡터 DB에 문서 추가
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("✅ 성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))

# 4. 검색 예시 함수
def search_products(query, n_results=3):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        print(f"🔍 [{idx}] {doc}")
        print(f"   가격: {meta['price']:,}원 | 브랜드: {meta['brand']}")
        print(f"   카테고리: {meta['category']}")
        print(f"   링크: {meta['link']}")
        print("-"*50)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ 성공적으로 100 개 상품 저장됨


In [4]:
# LLM 적용 전, 벡터 DB에 잘 들어가있는지 확인
search_products("피곤할 때 먹는 영양제", 5)

🔍 [1] 얼라이브 원스데일리 멀티/포맨/ 우먼 60정 비타민B군 미네랄 종합<b>영양제</b> 네츄럴라이프얼라이브 네이쳐스웨이 비타민제
   가격: 23,900원 | 브랜드: 네츄럴라이프얼라이브
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/309612084
--------------------------------------------------
🔍 [2] 베로카 발포비타민 60정 멀티 종합 비타민 <b>영양제</b> 물에타먹는 마시는 에너지 코스트코 베로카  비타민제
   가격: 23,700원 | 브랜드: 베로카
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9172711926
--------------------------------------------------
🔍 [3] 뉴트리포유 듀얼 이뮨샷 액상 마시는 멀티비타민 미네랄 <b>영양제</b> 30개입 피로회복 종합<b>영양제</b>  소셜빈 비타민제
   가격: 54,900원 | 브랜드: 
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/10532848731
--------------------------------------------------
🔍 [4] 센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b> 센트룸 화이자 비타민제
   가격: 34,900원 | 브랜드: 센트룸
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9244093197
--------------------------------------------------
🔍 [5] 세노비스 수험생 중학생 고등학생 성인 청소년 종합 <b>영양제</b> 멀티비타민 미네랄 젤리

In [26]:
# Hugging Face 및 Google API 인증
#from google.colab import userdata
#HF_TOKEN = userdata.get('HF_TOKEN') # Hugging Face 액세스 토큰
#GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Google Custom Search API 키
#GOOGLE_CSE_ID = userdata.get('GOOGLE_CSE_ID') # Programmable Search Engine ID
#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # OpenAI API Key


In [10]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    """
    구글 커스텀 검색을 실행하는 함수

    Parameters:
    - query (str): 검색할 질의어(검색어)입니다. 예: "비타민C 효능"
    - num (int): 검색 결과에서 반환할 최대 문서(스니펫) 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 검색 결과의 요약(snippet) 텍스트 리스트
    """
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]


In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

# 2. 검색 도구 정의 (Function Calling)
@tool
def search_products(query: str, max_results: int = 5) -> list:
    """제품 검색을 위해 Chroma DB를 쿼리하는 도구"""
    """
    Chroma DB를 쿼리하여 제품 정보를 검색하는 도구

    Parameters:
    - query (str): 제품 검색 질의어입니다. 예: "비타민C 고함량"
    - max_results (int): 반환할 최대 제품 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 제품의 정보(제품명, 가격, 링크)를 담은 딕셔너리들의 리스트
      예시: [{"제품": "제품명", "가격": 10000, "링크": "https://..."}, ...]
    """
    results = collection.query(
        query_texts=[query],
        n_results=max_results
    )
    return [{
        "제품": doc,
        "가격": meta['price'],
        "링크": meta['link']
    } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

@tool
def google_search_tool(query: str, num_results: int = 10) -> list:
    """구글 검색을 통해 제품 상세 정보 및 최신 논문을 찾는 도구. 질문 답변에 필요한 웹 기반 정보 수집에 사용"""
    """
    구글 검색 API를 통해 웹에서 정보를 검색하는 도구

    Parameters:
    - query (str): 검색할 질의어(검색어)입니다. 예: "2025년 영양제 트렌드"
    - num_results (int): 반환할 최대 검색 결과 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 검색 결과의 요약(snippet) 텍스트 리스트
    """
    return google_search(query, num=num_results)

# 3. 에이전트 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=OPENAI_API_KEY)
llm_with_tools = llm.bind_tools([search_products, google_search_tool])

system_prompt = """당신은 영양제 전문 약사 어시스턴트입니다. 다음 규칙을 준수하세요:
- 사용자 질문 유형에 따라 적절한 도구 선택
  - 제품 검색: search_products
  - 제품 상세 정보, 최신 논문: google_search_tool
- 검색 결과를 바탕으로 정확한 정보 제공
- 추천하는 이유를 자세하게 설명
- 가격, 성분 비교시 표 형태로 응답
- 모르는 정보는 확대 해석하지 말 것
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

tools = [search_products]
agent = create_openai_tools_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose=True 로 변경안해도 됨


In [ ]:
# 4. 실행 예시
response = agent_executor.invoke({
    "input": "종종 머리가 아프고 눈이 피곤해. 어떤 영양제를 먹으면 좋을지 추천해줘. 주요 성분과 함량도 알려줘."
})
print(response['output'])

두통과 눈 피로를 완화하기 위해 추천할 수 있는 영양제는 다음과 같습니다. 각 제품의 주요 성분과 함량도 함께 안내드립니다.

| 제품명 | 가격 | 주요 성분 | 링크 |
|--------|------|-----------|------|
| 종근당 아임비타 멀티비타민 이뮨플러스 14병 샷 | 21,900원 | 비타민 A, C, D, E, B군, 아연 | [구매하기](https://smartstore.naver.com/main/products/11097440087) |
| [1+1]종근당 아임비타 이뮨플러스 샷 2박스 | 23,500원 | 비타민 A, C, D, E, B군, 아연 | [구매하기](https://smartstore.naver.com/main/products/8566190007) |
| 뉴트리포유 듀얼 이뮨샷 액상 | 54,900원 | 비타민 B군, 비타민 C, 아연, 셀레늄 | [구매하기](https://smartstore.naver.com/main/products/10532848731) |
| 세노비스 수험생 중학생 고등학생 성인 청소년 종합 영양제 | 23,900원 | 비타민 A, C, D, E, B군, 아연 | [구매하기](https://smartstore.naver.com/main/products/7281402394) |
| 얼라이브 원스데일리 멀티 | 23,900원 | 비타민 A, C, D, E, B군, 미네랄 | [구매하기](https://smartstore.naver.com/main/products/309612084) |

### 추천 이유:
1. **비타민 B군**: 에너지 대사에 중요한 역할을 하며, 두통 완화에 도움을 줄 수 있습니다.
2. **비타민 C**: 항산화 작용을 통해 피로 회복에 기여합니다.
3. **아연**: 면역력 강화에 도움을 주며, 눈 건강에도 긍정적인 영향을 미칩니다.
4. **액상 형태**: 흡수율이 높아 빠르게 효과를 볼 수 있습니다.

이 제품들은 두통과 눈 피로를 완화하는 데 도움을 줄 수 있는 다양

In [ ]:
# 사용자 입력
user_symptom = input("🩺 증상이나 고민을 입력하세요: ")

# LLM 호출
response = agent_executor.invoke({
    "input": f"다음 증상에 맞는 영양제를 추천해줘: {user_symptom}. 결과는 표로 정리해줘. 제품명, 가격, 추천 이유, 주요 성분, 링크 포함해서."
})

# 결과 출력
print("\n🧠 최종 추천 결과 (표):\n")
print(response['output'])


김지은 블럭 세개로 UI향상및 질의응답 가능하게 넣었습니다.

In [17]:
!pip install ipywidgets
import sys
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [43]:
#식습관 분석, 불편현상에 대한 영양제 추천 답변 가능함
!pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import re

# 사용자 정보 입력 위젯
gender = widgets.ToggleButtons(
    options=["남성", "여성"],
    description='성별:',
    button_style='info'
)

age = widgets.IntText(
    value=30,
    description='만 나이:',
    layout=widgets.Layout(width='250px')
)

pregnant = widgets.ToggleButtons(
    options=["해당 없음", "임신 중", "임신 가능성"],
    description='임신 여부:',
    button_style='warning'
)

# 메뉴 및 입력창
menu = widgets.ToggleButtons(
    options=["식습관 분석", "불편 현상", "영양제 과다 복용 확인"],
    description='🩺 메뉴:',
    button_style='primary'
)

input_box = widgets.Text(
    placeholder='증상 또는 복용 제품명을 입력하세요.',
    description='입력:',
    layout=widgets.Layout(width='70%')
)

submit_btn = widgets.Button(description="🔍 제출", button_style="success")
output_box = widgets.Output()

# === 핵심 처리 로직 ===
def handle_submit(btn):
    output_box.clear_output()
    selected = menu.value
    user_input = input_box.value.strip()
    user_profile = {
        "성별": gender.value,
        "나이": age.value,
        "임신 여부": pregnant.value
    }

    with output_box:
        print(f"👤 사용자 정보: {user_profile}")
        print(f"🩺 선택한 기능: {selected}\n")

        # === 식습관 분석 ===
        if selected == "식습관 분석":
            print(f"🍽️ '{user_input}' 기반 식습관 분석 결과:")
            print("- 분석 결과에 따라 부족한 영양소는 '비타민 D', '철분'일 수 있습니다.")
            print("- 추천 영양제:")
            print("  • 비타민 D 1000IU")
            print("  • 철분 10~18mg")
            print("\n💡 ※ 해당 결과는 일반적인 권장사항이며, 개인 상태에 따라 달라질 수 있습니다.\n")

        # === 불편 증상 기반 추천 ===
        elif selected == "불편 현상":
            print(f"🤖 '{user_input}'에 대한 AI 추천 결과:")
            print("- 루테인, 오메가3, 비타민B군 제품이 적합할 수 있습니다.")
            print("\n※💡 이 답변은 빅데이터 기반으로 제공되며, 정확한 복약은 전문가 상담이 필요합니다.\n")

        # === 과다 복용 확인 ===
        elif selected == "영양제 과다 복용 확인":
            print("📦 입력한 복용 제품 목록:")
            print(user_input)

            # 간단한 철분 검출 (예: "철분" 키워드 포함된 제품 2개 이상)
            iron_related = re.findall(r'(철분|Iron)', user_input, flags=re.IGNORECASE)
            if len(iron_related) >= 2:
                print("⚠️ '철분' 성분이 여러 제품에 포함되어 있어 **과다 섭취 우려**가 있습니다.")
                print("※ 철분 일일 권장량: 여성 기준 **식사 포함 약 14~18mg**")
                print("\n📌 다음 제품은 중복될 수 있으므로 복용 제외를 고려해보세요:")
                print("  • 네이처메이드 철분 65mg")
                print("  • 솔가 철분 25mg")
                print("  • 페로그라드 F\n")
                print("※💡 과다 섭취는 개인 건강에 해로울 수 있으니, 전문가 상담을 권장드립니다.\n")
            else:
                print("✅ 특별한 과다 섭취 위험이 발견되지 않았습니다.")
                print("※ 단, 여러 제품 간 동일 성분 중복 여부를 개별 확인하시길 바랍니다.\n")

submit_btn.on_click(handle_submit)

# === UI 표시 ===
display(widgets.VBox([
    gender,
    age,
    pregnant,
    menu,
    input_box,
    submit_btn,
    output_box
]))


In [45]:
#수정이 필요합니다!!!!!!!!영양제 과다 복용확인칸에서 제대로 안되고 있습니다.

# 설치 필요 라이브러리:
# pip install langchain openai google-api-python-client ipywidgets

from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.tools import tool
from googleapiclient.discovery import build
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
import os

# --- 사용자 정보 위젯 ---
gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:', button_style='info')
age = widgets.IntText(value=30, description='만 나이:', layout=widgets.Layout(width='250px'))
pregnant = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:', button_style='warning')

menu = widgets.ToggleButtons(
    options=["식습관 분석", "불편 현상", "영양제 과다 복용 확인"],
    description='🩺 메뉴:', button_style='primary')
input_box = widgets.Text(placeholder='증상 또는 제품명을 입력하세요.', description='입력:', layout=widgets.Layout(width='70%'))
submit_btn = widgets.Button(description="🔍 제출", button_style="success")
output_box = widgets.Output()

# --- Google API 설정 (필요 시 입력) ---
GOOGLE_API_KEY = ""
CSE_ID = ""

def google_search(query, num=3):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=CSE_ID, num=num).execute()
    return "\n".join([item["snippet"] for item in res.get("items", [])])

@tool
def get_dietary_reference(query: str) -> str:
    """주어진 영양제 성분의 일일 권장량을 검색합니다."""
    return google_search(query + " 일일 권장량 site:nih.gov OR site:healthline.com")

# --- LangChain Agent ---
llm = OpenAI(openai_api_key="", temperature=0)
tools = [get_dietary_reference]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# --- 일일 권장량 기준 ---
reference_intake = {
    "철분": {"단위": "mg", "여성": {"default": 18, "임신 중": 27, "임신 가능성 있음": 18, "소아": 10}, "남성": {"default": 10, "소아": 10}, "UL": {"default": 45, "소아": 40}},
    "아연": {"단위": "mg", "여성": {"default": 8, "소아": 6}, "남성": {"default": 10, "소아": 6}, "UL": {"default": 40, "소아": 20}},
    "비타민C": {"단위": "mg", "여성": {"default": 75, "소아": 45}, "남성": {"default": 90, "소아": 45}, "UL": {"default": 650, "소아": 400}},
    "비타민D": {"단위": "IU", "여성": {"default": 600, "소아": 600}, "남성": {"default": 600, "소아": 600}, "UL": {"default": 4000, "소아": 2500}},
    "비타민A": {"단위": "μg", "여성": {"default": 700, "소아": 400}, "남성": {"default": 900, "소아": 400}, "UL": {"default": 3000, "소아": 900}},
    "칼슘": {"단위": "mg", "여성": {"default": 1000, "소아": 1300}, "남성": {"default": 1000, "소아": 1300}, "UL": {"default": 2500, "소아": 2000}},
    "오메가3": {"단위": "mg", "여성": {"default": 500, "소아": 250}, "남성": {"default": 500, "소아": 250}, "UL": {"default": 3000, "소아": 1000}},
    "비타민B12": {"단위": "μg", "여성": {"default": 2.4, "소아": 1.2}, "남성": {"default": 2.4, "소아": 1.2}, "UL": {"default": None, "소아": None}}
}

# --- 단위 변환 함수 ---
def normalize_unit(value, unit):
    if unit.lower() in ["mg"]: return float(value)
    if unit.lower() in ["μg", "mcg"]: return float(value) / 1000
    if unit.lower() in ["g"]: return float(value) * 1000
    if unit.upper() == "IU": return float(value)
    return 0

# --- 분석 로직 ---
def check_overdose(user_input, profile):
    result = []
    age = profile['나이']
    gender = profile['성별']
    pregnancy = profile['임신 여부']
    found = False

    for nutrient, data in reference_intake.items():
        pattern = fr"{nutrient}[\s\S]*?(\d+)(mg|IU|μg|mcg|g)"
        matches = re.findall(pattern, user_input, re.IGNORECASE)
        total = sum(normalize_unit(val, unit) for val, unit in matches)
        if total == 0: continue

        ref = data[gender].get("소아") if age < 18 else data[gender].get(pregnancy, data[gender].get("default"))

        ul = data.get("UL")
        if isinstance(ul, dict):
            ul = ul.get("소아") if age < 18 else ul.get(pregnancy, ul.get("default"))

        unit_label = data['단위']

        if ul and total > ul:
            found = True
            result.append(f"⚠️ '{nutrient}' 총 섭취량 {total:.1f}{unit_label} > UL({ul}{unit_label}) → **과다 섭취 경고**")
            result.append(f"→ {nutrient} 권장량: {ref}{unit_label}, 최대: {ul}{unit_label}")
        else:
            result.append(f"✅ '{nutrient}' 섭취량 {total:.1f}{unit_label} (UL: {ul or '정보 없음'})")

    if not found:
        result.append("✅ 과다 섭취된 성분은 발견되지 않았습니다.")

    result.append("※ 개인차가 있으므로 전문가 상담을 권장드립니다.")
    return "\n".join(result)

# --- 제출 버튼 이벤트 ---
def handle_submit(btn):
    output_box.clear_output()
    profile = {"성별": gender.value, "나이": age.value, "임신 여부": pregnant.value}
    selected = menu.value
    user_input = input_box.value.strip()

    with output_box:
        print(f"👤 사용자 정보: {profile}")
        print(f"🩺 선택한 기능: {selected}\n")

        if selected == "영양제 과다 복용 확인":
            print("📦 입력 제품:", user_input)
            result = check_overdose(user_input, profile)
            print(result)
        else:
            print("🔧 이 기능은 현재 데모용으로 과다 섭취 확인만 지원됩니다.")

submit_btn.on_click(handle_submit)

# --- UI 출력 ---
display(widgets.VBox([gender, age, pregnant, menu, input_box, submit_btn, output_box]))
